# Parallel programming code

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 18.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=49c90e8c3e118c8165662310f5a5b292cb8df16a26c3cea6f906538a2e7abb7c
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [2]:
# Press Shift+F10 to execute it or replace it with your code.
# Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.
from pyspark import SparkContext

In [3]:
# Press the green button in the gutter to run the script.
if __name__ == '__main__':
    # create a SparkContext
    sc = SparkContext("local", "CheapestOperator")

    # load the price lists as RDDs
    operator_a = sc.textFile("./op_a.txt").map(lambda line: line.split())
    operator_b = sc.textFile("./op_b.txt").map(lambda line: line.split())

    # create a dictionary to store the prices for each prefix and operator
    prefix_prices_op = {}

    # add prefix_prices_op from operator A
    for prefix, price in operator_a.collect():
        prefix_prices_op[prefix] = (float(price),'operator A')

    # update prefix_prices_op with operator B (taking new prefixes and skiping duplicates, with the goal of keeping only the cheapest values)
    for prefix, price in operator_b.collect():
        if prefix in prefix_prices_op:
            min_price = min(prefix_prices_op[prefix][0], float(price)) 
            prefix_prices_op[prefix] = ( min_price, 'operator B')
        else:
            prefix_prices_op[prefix] = (float(price), 'operator B')

    # define a function to find the cheapest operator for a given telephone number which has longest prefix
    def cheapest_operator(number):
        cheapest_price = float('inf')
        cheapest_operator = None
        longest_prefix = 0
        for prefix in prefix_prices_op.keys():
            if number.startswith(prefix):
              if len(prefix) >= longest_prefix:
                longest_prefix = len(prefix)
                cheapest_price = prefix_prices_op[prefix][0] # Get price
                cheapest_operator = prefix_prices_op[prefix][1] #Get operator
              
        return cheapest_operator, cheapest_price


    # test the function
    number = "4673212345"
    print(cheapest_operator(number))  # prints Cheapest operator and its price "Operator A 1.1"

    # stop the SparkContext
    sc.stop()

('operator A', 1.1)
